In [ ]:
import json
from datetime import datetime
import pandas as pd

In [ ]:
!pip install transformers

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read)

In [ ]:
import json
import re
import pandas as pd

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv("replace_with_your_file.csv")

def clean_text(text):
    # Convert the input to a string if it's not already
    text = str(text)  # This will handle integers and other data types

    # Remove HTML tags
    text = re.sub(r"<.*?>", "", text)

    # Remove URLs
    text = re.sub(r"https?://\S+|www\.\S+", "", text)

    # Remove special characters (keep only alphanumeric and whitespace)
    text = re.sub(r"[^\w\s]", "", text)

    # Handle missing values (replace with empty string)
    text = text if text else ""

    return text

def preprocess_text(text):
  return clean_text(text)

# Convert to JSONL format with preprocessing:
with open("replace_with_fine_tuning_data.jsonl", "w") as f:
    for index, row in df.iterrows():
        # Assuming your CSV has columns named "instruction" and "output"
        instruction = row["text"]
        output = row["label"]

        # Check for missing values
        if pd.isnull(instruction) or pd.isnull(output):
            continue  # Skip this row if either is missing

        json_object = {
            "messages": [
                {"role": "system", "content": "You are a helpful and harmless AI assistant."},
                {"role": "user", "content": preprocess_text(instruction)},
                {"role": "assistant", "content": preprocess_text(output)},
            ]
        }
        f.write(json.dumps(json_object) + "\n")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "openai-community/gpt2"  # Repo ID

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# ... (Use the model and tokenizer for inference or fine-tuning) ...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import json

def extract_text_from_jsonl(file_path):
  text_data = []
  with open("replace_with_fine_tuning_data.jsonl", "r") as f:
    for line in f:
                 data = json.loads(line)
                 text_data.append(data["messages"][1]["content"] + " " + data["messages"][2]["content"])  # Concatenate user and assistant messages
  return text_data

text_data = extract_text_from_jsonl("replace_with_fine_tuning_data.jsonl")

In [ ]:
!pip install -q bitsandbytes accelerate transformers peft
!pip install -U bitsandbytes
!pip install datasets transformers


In [ ]:
from transformers import TrainingArguments, Trainer, AutoModelForCausalLM
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset
import torch
from peft import LoraConfig, get_peft_model

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

# Load the base model with bitsandbytes for quantization
model = AutoModelForCausalLM.from_pretrained(
    "openai-community/gpt2",
    load_in_8bit=True,  # Enable 8-bit quantization
    device_map="auto",  # Automatically assign to the appropriate device
)


# Define the LoRA configuration
lora_config = LoraConfig(
    r=8,  # Rank of the LoRA matrices
    lora_alpha=16,  # Scaling factor for the LoRA matrices
    lora_dropout=0.05,  # Dropout probability for the LoRA layers
    bias="none",  # Bias type for the LoRA layers
    task_type="CAUSAL_LM",  # Task type for fine-tuning
    fan_in_fan_out=True # Explicitly setting fan_in_fan_out to True to suppress the warning
)

# Apply LoRA to the base model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Print the number of trainable parameters

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,  # Reduce batch size if needed
    per_device_eval_batch_size=4,  # Reduce batch size if needed
    num_train_epochs=3,
    logging_dir="./logs",
    fp16=True,  # Enable mixed precision training if supported by your hardware
    gradient_accumulation_steps=4,  # Increase gradient accumulation steps if needed
)

# Load the text data you extracted
# Assuming 'text_data' is a list of strings from your 'extract_text_from_jsonl' function
# Create a Hugging Face Dataset from your text data
train_dataset = Dataset.from_dict({"text": text_data})

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize the dataset
tokenized_datasets = train_dataset.map(tokenize_function, batched=True)


# Access the 'train' split of the dataset
train_dataset = tokenized_datasets

# Add the labels to the inputs
def prepare_inputs_for_training(examples):
    examples["labels"] = [[-100] + x[:-1] for x in examples["input_ids"]]
    return examples

train_dataset = train_dataset.map(
    prepare_inputs_for_training,
    batched=True,
    remove_columns=["text"],
)

# Create the Trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

# Save the model
trainer.save_model("./fine_tuned_gpt2")

In [ ]:
import pandas as pd

df_validation = pd.read_csv("replace_with_validation.csv")

# Assuming your CSV has columns named "text" (for prompts) and "label" (for expected outputs)
prompts = df_validation["text"].tolist()
expected_outputs = df_validation["text"].tolist()

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "/content/fine_tuned_gpt2"
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")  # Load the tokenizer using the original model ID
model = AutoModelForCausalLM.from_pretrained(model_id)

# Set the padding token after loading the tokenizer
tokenizer.pad_token = tokenizer.eos_token

inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

generated_outputs = []
for prompt in prompts:
    generated_output = generator(prompt, max_length=100, num_return_sequences=1)[0]['generated_text']  # Adjust parameters as needed
    generated_outputs.append(generated_output)

In [32]:
!pip install nltk  # Install NLTK if you haven't already
import nltk
from nltk.translate.bleu_score import sentence_bleu

bleu_scores = []
for generated, expected in zip(generated_outputs, expected_outputs):
    # Convert 'expected' to a string before splitting
    bleu_score = sentence_bleu([str(expected).split()], generated.split())  # Calculate BLEU score
    bleu_scores.append(bleu_score)

average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU score: {average_bleu}")

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU score: 0.9691038014873151


In [33]:
!pip install rouge-score
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calculate and print ROUGE scores for all generated outputs
for generated_output, expected_output in zip(generated_outputs, expected_outputs):
  scores = scorer.score(generated_output, str(expected_output))  # Convert expected_output to string
  print(f"Generated Output: {generated_output}")
  print(f"Expected Output: {expected_output}")
  print(f"ROUGE Scores: {scores}")
  print("-" * 20)  # Print a separator between examples

Streaming output truncated to the last 5000 lines.
Expected Output: im feeling particularly smug create my own
ROUGE Scores: {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
--------------------
Generated Output: i feel it more when i see you not bothered
Expected Output: i feel it more when i see you not bothered
ROUGE Scores: {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
--------------------
Generated Output: i don t want you my reader friends to feel like you need to feel sorry for me
Expected Output: i don t want you my reader friends to feel like you need to feel sorry for me
ROUGE Scores: {'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=

In [34]:
!pip install nltk
import nltk
nltk.download('wordnet')  # Download WordNet for METEOR

from nltk.translate.meteor_score import meteor_score

meteor_score_value = meteor_score([expected_output.split()], generated_output.split())  # Replace with your actual outputs
print(meteor_score_value)

0.9999142661179699


[nltk_data] Downloading package wordnet to /root/nltk_data...
